<a href="https://colab.research.google.com/github/mulumbi/Spotify_Data_Analysis/blob/main/spotify_anal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for maths
import numpy as np
import pandas as pd
import scipy.stats as stats
import json

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
from matplotlib.pyplot import figure
import datetime as dt

% matplotlib inline

In [11]:
# loading the different datasets

streams1 = pd.read_json("/content/StreamingHistory0.json", orient='records')
streams2 = pd.read_json("/content/StreamingHistory1.json", orient='records')
streams3 = pd.read_json("/content/StreamingHistory2.json", orient='records')
stream_frames = [streams1, streams2, streams3]
streams = pd.concat(stream_frames)
streams.head()


,endTime,artistName,trackName,msPlayed
0,2021-07-18 22:11,Selena Gomez,Back To You,1582
1,2021-07-18 22:11,Lauv,Modern Loneliness,225776
2,2021-07-18 22:11,Adele,Send My Love (To Your New Lover),1951
3,2021-07-28 00:00,James Carter,Used To,159258
4,2021-07-28 00:03,James Carter,Used To,22527684


In [4]:
# converting the df from long to wide
grouped_df = streams.groupby(["trackName", "artistName"])
song_stream_times = grouped_df["msPlayed"].apply(list)
song_stream_times = song_stream_times.reset_index()



# sum up total stream minutes per song and sort in descending order
song_stream_times["msPlayed"] = song_stream_times["msPlayed"].map(lambda msPlayed: sum(msPlayed)//3600000)
cleaned_stream_times =song_stream_times.query("msPlayed > 6")
cleaned_stream_times.rename(
    columns={"msPlayed":"hrsPlayed"}
          ,inplace=True)
cleaned_stream_times.sort_values("hrsPlayed", ascending=False)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,trackName,artistName,hrsPlayed
6330,mirrorball,Taylor Swift,116
3668,Nico's Red Truck,Dijon,86
4998,Style,Taylor Swift,84
5637,Used To,James Carter,70
6232,emily,Jeremy Zucker,64
...,...,...,...
1121,Cool Again,Shoffy,7
3461,Monster,Kanye West,7
3574,NDOVU NI KUU,NDOVU KUU,7
6228,eMcimbini (Live),Kabza De Small,7


Next steps:


*   create another df grouping the individual stream dates
*   Join the two dfs to create a wide version of the long df we originally had

*   Filter out any tracks that I listened to for less than 2 hours
*   visualize my top artists and bottom artists in a time vs artist graph, where the data points are individual songs (with labels)

*   do the same with my bottom 50 artists
*   create a streaming timeline per artist using dates (per month) ie

 *  see if I binge listen to an artist on a day vs if I shuffle through multiple in a session

 *   see if theres artists I listen to everyyday/ frequently and the habits ie do I listen to some artists on weekdays vs weekends









In [5]:
# creating a list of unique artists and creation of an artist class


artist_list = list(set(cleaned_stream_times["artistName"].tolist()))
# remove podcasts from the list
artist_list.remove('The Afrobeats Podcast')
artist_list.remove('THE GOOD, THE BAD AND THE BUSHERIAN ' )


Artist Class
Members:


*   List of songs
*   List listening times
*   Total listening time

Song Class
Members:

*   Total listening time
*   List of map of day of the week to number of streams during those days
*   Average time of day I listened to the song


In [18]:
# creating artist and song class
class Artist:
    name = ""
    song_titles = []
    songs = []
    listening_times = []
    total_times = 0


    def __init__(self):
        pass
      
   #def __init__(self, song, time):
     # if song not in self.songs:
       # self.songs.append(song)
      #  self.listening_times.append(time)


class Song:
    title = ""
    avg_listening_time = '00:00:00'
    dates = []
    durations = []
    total_time = 0


    def __init__(self):
        pass
      
    #def __init__(self, song, time):
     # self.title = song
     # self.dates.append(time)

    def avg_time(times):
      avg = 0
      for elem in times:
          avg += elem.second + 60*elem.minute + 3600*elem.hour
      avg /= len(times)
      rez = str(avg/3600) + ' ' + str((avg%3600)/60) + ' ' + str(avg%60)
      return dt.datetime.strptime(rez, "%H %M %S")

In [24]:
# populating the classes
Artists = []
Songs = []



grouped_df.head()
for musician in artist_list:
  curr = Artist()
  curr.name = musician
  # get all songs by that artist
  all_songs = streams.loc[streams['artistName'] == musician]

  #looping through all song records for a particular artist
  for ind, row in all_songs.iterrows():
    song_record = all_songs.loc[ind]
    title = song_record["trackName"]
    curr_song = None

    # if the track isnt already in our artist's list
    if not title in curr.song_titles:
      curr.song_titles.append(title)
      curr_song = Song()
      curr_song.title = title

      # update the song's play date and duration played
      curr_song.dates.append(song_record["endTime"])
      curr_song.durations.append(song_record["msPlayed"])

      # getting the total stream time
      total_plays_record = cleaned_stream_times.loc[(cleaned_stream_times['artistName'] == musician) &
                             (cleaned_stream_times['trackName'] == title)]
      curr_song.total_time = total_plays_record["hrsPlayed"]
      curr.songs.append(curr_song)

    # if the song is already in our artist list
    else:
      curr_song = curr.songs[curr.song_titles.index(title)]
      curr_song.dates.append(song_record["endTime"])
      curr_song.durations.append(song_record["msPlayed"])




  Artists.append(curr)


print(Artists)

ValueError: ignored